In [21]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.feature_selection import RFE, RFECV 

from precision_recall_cutoff import precision_recall_cutoff

#defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'rachaeld-data445'
bucket = s3.Bucket(bucket_name)

#defining the csv file
file_key = 'turnover.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

turnover = pd.read_csv(file_content_stream)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [22]:
## Changing sales and salary to dummies
turnover = pd.concat([turnover.drop(columns = ['sales', 'salary'], axis = 1), pd.get_dummies(turnover[['sales', 'salary']])], axis = 1)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,...,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [23]:
##engineering features for the decision tree model
turnover['interaction_1'] = np.where(((turnover['satisfaction_level'] >= .115) & 
                                     (turnover['satisfaction_level'] <= .465) & 
                                     (turnover['number_project'] > 2.5)), 1, 0)

turnover['interaction_2'] = np.where(((turnover['satisfaction_level'] <= .465) & 
                                     (turnover['number_project'] <= 2.5) & 
                                     (turnover['last_evaluation'] <= .575)), 1, 0)

turnover['interaction_3'] = np.where(((turnover['satisfaction_level'] > .465) & 
                                     (turnover['time_spend_company'] <= 4.5) & 
                                     (turnover['average_montly_hours'] <= 290.5)), 1, 0)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,...,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium,interaction_1,interaction_2,interaction_3
0,0.38,0.53,2,157,3,0,1,0,0,0,...,0,1,0,0,0,1,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,0.11,0.88,7,272,4,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
3,0.72,0.87,5,223,5,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
4,0.37,0.52,2,159,3,0,1,0,0,0,...,0,1,0,0,0,1,0,0,1,0


In [24]:
##defining input and target variables
X = turnover.drop(columns = 'left', axis = 1)
Y = turnover['left']

##splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2, stratify = Y)

In [25]:
## RFE with Random forest ###
##model
RF_rfe = RFE(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3), n_features_to_select = 5).fit(X_train, Y_train)

##extracting the feature names
print(X_train.columns[RF_rfe.support_])

Index(['satisfaction_level', 'time_spend_company', 'interaction_1',
       'interaction_2', 'interaction_3'],
      dtype='object')


In [26]:
##defining input and target 
X_train_1 = X_train[['satisfaction_level', 'time_spend_company', 'interaction_1',
       'interaction_2', 'interaction_3']]
X_test_1 = X_test[['satisfaction_level', 'time_spend_company', 'interaction_1',
       'interaction_2', 'interaction_3']]

##buolding the model
RF_md_1 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train_1, Y_train)

##predicting on test
RF_pred_1 = RF_md_1.predict_proba(X_test_1)[:, 1]

##changing liklihoods to labels 
RF_labels_1 = precision_recall_cutoff(Y_test, RF_pred_1)

print(classification_report(Y_test, RF_labels_1))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      2286
           1       0.94      0.87      0.91       714

    accuracy                           0.96      3000
   macro avg       0.95      0.93      0.94      3000
weighted avg       0.96      0.96      0.96      3000



In [27]:
### RFECV with Random Forest ###

auto_feature_selection = RFECV(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3), step = 1, min_features_to_select = 2, cv = 3).fit(X_train, Y_train)

##extracting feature names
print(X_train.columns[auto_feature_selection.support_])

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'interaction_1', 'interaction_2', 'interaction_3'],
      dtype='object')


In [28]:
##defining the input and target
X_train_2 = X_train[['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'interaction_1', 'interaction_2', 'interaction_3']]

X_test_2 = X_test[['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'interaction_1', 'interaction_2', 'interaction_3']]

##building the model
RF_md_2 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train_2, Y_train)

##predicting on test
RF_pred_2 = RF_md_2.predict_proba(X_test_2)[:, 1]

##changing liklihoods to labels 
RF_labels_2 = precision_recall_cutoff(Y_test, RF_pred_2)

print(classification_report(Y_test, RF_labels_2))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2286
           1       0.95      0.92      0.93       714

    accuracy                           0.97      3000
   macro avg       0.96      0.95      0.96      3000
weighted avg       0.97      0.97      0.97      3000



In [ ]:
## Based on the above results I would go with the second random forest model, because it has better values. 